$$d_1 = \frac{ln\frac{S_0}{X}+(r+\frac{\sigma ^2}{2})T}{\sigma \sqrt{T}}$$
$$d_2 = \frac{ln\frac{S_0}{X}+(r-\frac{\sigma ^2}{2})T}{\sigma \sqrt{T}}$$
$$ C_0 = S_0N(d_1) - Ke^{-rT}N(d_2) $$

In [2]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import pandas_datareader.data as web 

def d1(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
    return d1
def d2(S, K, T, r, sigma):
    d2 = (np.log(S/K) + (r - sigma**2 / 2) * T) / (sigma * np.sqrt(T))
    return d2
def Call(S, K, T, r, sigma):
    Call = S * norm.cdf(d1(S, K, T, r, sigma)) - K * np.e ** (-r*T) * norm.cdf(d2(S, K, T, r, sigma))
    return Call

df = web.DataReader('WMT', 'yahoo','2020-03-24', '2021-03-26')
df['logreturn'] = np.log (df['Close'] / df['Close'].shift(1))

sigma = np.sqrt(252) * df['logreturn'].std()
TB = web.DataReader('^TNX', 'yahoo','2020-03-24', '2021-03-26' ) #Treasury Bil yield data

r = TB.Close.iloc[-1] / 100 # risk free interst rate
S = df.Close.iloc[-1] # Current Price == Last Close Price
K = 135  # Strike Price
T = 35/365 # Option Expires at April 1 --> 35 days
print('By the Black-Scholes Model, price of the Call Option is:', end = ' ')
print(Call(S, K, T, r, sigma))

By the Black-Scholes Model, price of the Call Option is: 4.115553973100759


In [4]:
print(S, K, T, r, sigma)

135.1300048828125 135 0.0958904109589041 0.01659999966621399 0.2364207716796307


In [2]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import pandas_datareader.data as web 

def d1(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r -delta + 0.5*sigma**2) * T) / (sigma * np.sqrt(T))
    return d1
def d2(S, K, T, r, sigma):
    d2 = (np.log(S/K) + (r -delta - 0.5*sigma**2) * T) / (sigma * np.sqrt(T))
    return d2
def Call(S, K, T, r, sigma):
    return S * np.e**(-delta*T)* norm.cdf(d1(S, K, T, r, sigma))  - K * np.e ** (-r*T) * norm.cdf(d2(S, K, T, r, sigma))
def Put(S, K, T, r, sigma):
    return  -S* np.e**(-delta*T) *norm.cdf(-d1(S, K, T, r, sigma)) + K * np.e ** (-r*T) * norm.cdf(-d2(S, K, T, r, sigma))

In [3]:
r = 0.0516
S =143.13
K = 140
T = 42/365
sigma = 0.28
delta = 0

print(Call(S, K, T, r, sigma))
print(Put(S, K, T, r, sigma))

7.557097856756798
3.5983059871633074


In [14]:
norm.cdf(d2(S, K, T, r, sigma)) * np.e**(-r)

0.784982946485151

In [12]:
norm.cdf(d1(S, K, T, r, sigma))*S*np.e**(-delta)

236.9317804263279

In [17]:
Call(S, K, T, r, sigma) + 0.5*Put(S, K, T, r, sigma)

5.296696925002253

In [6]:
8*np.e**((8/12)*0.05)

8.271160908108593

In [1]:
def get_price_w_rf(right,T,S,X,v,rf,N=100000):
    D = np.exp(-rf*(T/365))
    prices = np.cumprod(1+(np.random.randn(T,N) * v/ np.sqrt(365)), axis=0)*S 
    if right == 'c':
        return np.sum((prices[-1,:]-X*D)[prices[-1,:]>X*D])/prices.shape[1]
    else:
        return -np.sum((prices[-1,:]-X*D)[prices[-1,:]<X*D])/prices.shape[1]

get_price_w_rf('c', 35,135.13 ,135 ,0.2364207716796307,0.01659)

4.113106599065805

In [2]:
import numpy as np

S= 135.13
T= 35/365
r= 0.01659
sigma= 0.2364207716796307
Nsimulations= 5000
Nsteps= 250
K= 135

dt= T/Nsteps

drift= (r-(sigma**2)/2)*dt
a= sigma*np.sqrt(dt)
x= np.random.normal(0,1,(Nsimulations, Nsteps))


Smat= np.zeros((Nsimulations, Nsteps))
Smat[:, 0] += S

for i in range(1, Nsteps):
    Smat[:, i] += Smat[:, i-1]*np.exp(drift +a*x[:, i])


q= Smat[:, -1]- K
for i in range(len(q)):
    if q[i]< 0:
        q[i]= 0
    else:
        q[i]= q[i]

p= K- Smat[:, -1]
for i in range(len(p)):
    if p[i]< 0:
        p[i]= 0
    else:
        p[i]= p[i]

payoff_call= np.mean(q)
payoff_put= np.mean(p)

call= payoff_call*np.exp(-r*T)
put= payoff_put*np.exp(-r*T)

call

4.018701937635613